In [1]:
from anthropic import Anthropic
from anthropic.types import ToolParam

In [2]:
# Load environment variables
from helper import load_env
load_env()

MODEL_NAME="claude-3-5-sonnet-20241022"
client = Anthropic()


In [3]:
def get_weather(location):
    if not location or location.strip() == "":
        raise ValueError('Location cannot be empty')
    
    # Normalize location for consistent sample data
    location = location.strip().lower()
    
    # Sample weather data for different locations
    sample_weather_data = {
        "new york": {
            "current": {"temp_c": 22, "condition": {"text": "Partly cloudy"}, "humidity": 65, "wind_mph": 8.5}
        },
        "london": {
            "current": {"temp_c": 15, "condition": {"text": "Overcast"}, "humidity": 78, "wind_mph": 12.3}
        },
        "tokyo": {
            "current": {"temp_c": 28, "condition": {"text": "Sunny"}, "humidity": 55, "wind_mph": 6.2}
        },
        "sydney": {
            "current": {"temp_c": 19, "condition": {"text": "Light rain"}, "humidity": 82, "wind_mph": 15.7}
        },
        "paris": {
            "current": {"temp_c": 17, "condition": {"text": "Cloudy"}, "humidity": 71, "wind_mph": 9.8}
        }
    }
    
    # Check if we have specific data for this location
    for location_data, temperature_data in sample_weather_data.items():
        if location_data == location:
            output = f"{location} has temperature of {temperature_data['current']['temp_c']}"
            return output

    return None


# Tool schema (JSON-serializable)
get_weather_schema: ToolParam = {
    "name": "get_weather",
    "description": "Get current weather information for a specified location. Returns temperature and weather conditions for major cities including New York, London, Tokyo, Sydney, and Paris.",
    "input_schema": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The location/city name to get weather information for. Supports major cities like New York, London, Tokyo, Sydney, Paris."
            }
        },
        "required": ["location"]
    }
}

In [4]:
# Let ask Claude for temperature in Paris.
messages = list()

messages.append({
    "role": "user",
    "content": "What is the current weather in Paris"
})


response = client.messages.create(
    model=MODEL_NAME,
    max_tokens=1000,
    messages=messages,
    tools=[get_weather_schema],
)

In [12]:
# output from LLM for tool use.
print(type(response))
print(f'Claude Full reponse: {response}')
print('\n')

print(f'Claude stop reason: {response.stop_reason}')
print('\n')

print(type(response.content[0]))
print(f'Claude Response with TextBlock: {response.content[0]}')
print('\n')

print(type(response.content[1]))
print(f'Claude Response with ToolUseBlock: {response.content[1]}')
print('\n')


print(type(response.content[1].id))
print(f'Claude Response with ToolUseBlock with ToolID: {response.content[1].id}')
print('\n')

print(type(response.content[1].name))
print(f'Claude Response with ToolUseBlock with ToolName: {response.content[1].name}')
print('\n')

print(type(response.content[1].input))
print(f'Claude Response with ToolUseBlock with ToolInput: {response.content[1].input}')
print('\n')

<class 'anthropic.types.message.Message'>
Claude Full reponse: Message(id='msg_01HDb3cH9rfv2PYUjCiqDqfH', content=[TextBlock(text="I'll help you check the current weather in Paris using the get_weather function.", type='text'), ToolUseBlock(id='toolu_01Whg5QRhLJ6mWqQaKwTr9vi', input={'location': 'Paris'}, name='get_weather', type='tool_use')], model='claude-3-5-sonnet-20241022', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=435, output_tokens=71, cache_creation={'ephemeral_5m_input_tokens': 0, 'ephemeral_1h_input_tokens': 0}, service_tier='standard'))


Claude stop reason: tool_use


<class 'anthropic.types.text_block.TextBlock'>
Claude Response with TextBlock: TextBlock(text="I'll help you check the current weather in Paris using the get_weather function.", type='text')


<class 'anthropic.types.tool_use_block.ToolUseBlock'>
Claude Response with ToolUseBlock: ToolUseBlock

In [13]:
# add the above message back to messages list.

messages.append({
    "role": "assistant",
    "content": response.content
})



In [14]:
messages

[{'role': 'user', 'content': 'What is the current weather in Paris'},
 {'role': 'assistant',
  'content': [TextBlock(text="I'll help you check the current weather in Paris using the get_weather function.", type='text'),
   ToolUseBlock(id='toolu_01Whg5QRhLJ6mWqQaKwTr9vi', input={'location': 'Paris'}, name='get_weather', type='tool_use')]}]

In [15]:
# call the Tool use with the LLM Output.
messages.append({
    "role": "user",
    "content": [{
        "type": "tool_result",
        "tool_use_id": response.content[1].id,
        "content": get_weather(**response.content[1].input),
        "is_error": False
    }]
})

messages


[{'role': 'user', 'content': 'What is the current weather in Paris'},
 {'role': 'assistant',
  'content': [TextBlock(text="I'll help you check the current weather in Paris using the get_weather function.", type='text'),
   ToolUseBlock(id='toolu_01Whg5QRhLJ6mWqQaKwTr9vi', input={'location': 'Paris'}, name='get_weather', type='tool_use')]},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'toolu_01Whg5QRhLJ6mWqQaKwTr9vi',
    'content': 'paris has temperature of 17',
    'is_error': False}]}]

In [16]:
# Final call to LLM with tool output.

response = client.messages.create(
    model=MODEL_NAME,
    max_tokens=1000,
    messages=messages,
    tools=[get_weather_schema]
)

In [19]:
# output from LLM for tool use.
print(type(response))
print(f'Claude Full reponse: {response}')
print('\n')

print(f'Claude stop reason: {response.stop_reason}')
print('\n')

print(type(response.content[0]))
print(f'Claude Response with TextBlock: {response.content[0]}')
print('\n')


print(f'Claude Final Output: {response.content[0].text}')
print('\n')


<class 'anthropic.types.message.Message'>
Claude Full reponse: Message(id='msg_01LgdguFi8NpetiwRwkbWf1D', content=[TextBlock(text='According to the current weather data, the temperature in Paris is 17 degrees (Celsius).', type='text')], model='claude-3-5-sonnet-20241022', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=523, output_tokens=24, cache_creation={'ephemeral_5m_input_tokens': 0, 'ephemeral_1h_input_tokens': 0}, service_tier='standard'))


Claude stop reason: end_turn


<class 'anthropic.types.text_block.TextBlock'>
Claude Response with TextBlock: TextBlock(text='According to the current weather data, the temperature in Paris is 17 degrees (Celsius).', type='text')


Claude Final Output: According to the current weather data, the temperature in Paris is 17 degrees (Celsius).


